**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/p5q7gzupa2ndw55/sigmoid_regression_data.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Sigmoidná regresia pomocou autodiff-u z PyTorch-u

Keď sme už ukázali ako v rámci PyTorch funguje automatická diferenciácia (autodiff), môžeme sa ju teraz pokúsiť aplikovať na sigmoidnú regresiu, ktoré sme už raz vyriešili pomocou symbolických gradientov. Začneme načítaním príslušnej dátovej množiny z CSV súboru.



In [ ]:
#@title -- Loading and Preprocessing the Data: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

df_train, df_test = train_test_split(
    df, stratify=y_stratify, test_size=0.3, random_state=4)

plt.scatter(df_train['x'], df_train['y'], marker='x', label="training data")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

categorical_inputs = []
numeric_inputs = ['x']
output = 'y'

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[[output]].values

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[[output]].values

Ako vieme, PyTorch operuje s tenzormi a nie s klasickými poľami, preto potrebujeme dáta najprv vhodne obaliť.



In [ ]:
X_train_t = torch.as_tensor(X_train)
Y_train_t = torch.as_tensor(Y_train)
X_test_t = torch.as_tensor(X_test)
Y_test_t = torch.as_tensor(Y_test)

### Sigmoidná funkcia

Pripomeňme, že sigmoidná funkcia je definovaná nasledovne:
\begin{equation}
\sigma(x) = \frac{1}{1 + e^{-x}}.
\end{equation}

Keďže naša sigmoida môžem byť posunutá alebo môže mať inú strmosť, aplikujeme na jej vstup najprv lineárnu transformáciu, ktorej parametre $a$ a $c$ sa naučíme z dát. Náš úplný regresný model bude teda vyzerať takto:
\begin{align}
u &= ax + c \
\sigma(u) &= \frac{1}{1 + e^{-u}}.
\end{align}

Alebo ak všetko zložíme do jednej funkcie:
\begin{equation}
f(x, a, c) = \frac{1}{1 + e^{-ax - c}}
\end{equation}

Zadefinujme si teraz náš regresný model pomocou PyTorch operácií.



In [ ]:
def sigmoid_model(X, a, c):
    return torch.sigmoid(X*a + c)

### Chybová funkcia a premenné

Ako chybovú funkciu použijeme strednú kvadratickú chybu. Definovať ju môžeme takto:



In [ ]:
def compute_loss(Y, y):
    return ((y - Y)**2).mean()

Potrebujeme tiež vytvoriť premenné `a` a `c` (tenzory obalíme ako premenné, pretože budeme ich hodnoty z kódu aktualizovať) a určiť rýchlosť učenia.



In [ ]:
a = Variable(torch.as_tensor(np.random.uniform(0, 1)), requires_grad=True)
c = Variable(torch.as_tensor(np.random.uniform(0, 1)), requires_grad=True)
learning_rate = 0.1

### Optimalizácia metódou klesajúceho gradientu

Ďalej môžeme napísať slučku, ktorá náš model zoptimalizuje metódou klesajúceho gradientu. Pripomeňme, že gradient sa dá vyrátať jednoducho spustením modelu dopredne a následným volaním metódy `backward()` výstupe – v našom prípade na chybe `loss`.

Musíme si však dať pozor, aby:

* Sme prestali sledovať gradienty (pomocou `with torch.no_grad():`) keď aktualizujeme parametre `a` a `c`: inak sa PyTorch bude snažiť aj tieto operácie začleniť do výpočtového grafu, čo skončí chybou.
* Sme po každej epoche vynulovali gradienty všetkých premenných. Keďže gradienty sa kumulujú, v opačnom prípade by sa nové gradienty len pridali ku hodnotám z predošlej epochy.


In [ ]:
for epoch in range(2500):
    y = sigmoid_model(X_train_t, a, c)
    loss = compute_loss(Y_train_t, y)
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        c -= learning_rate * c.grad
        
    a.grad.zero_()
    c.grad.zero_()
    
    if epoch % 100 == 0:
        print("Epoch {}; loss {}.".format(epoch, loss.item()))

Takto sme získali hodnoty `a` a `c`.



In [ ]:
print("a = {}\nc = {}\nloss = {}".format(
    a.item(), c.item(), loss.item()))

Pozrime sa, ako bude vyzerať naša regresná krivka.



In [ ]:
xx = torch.linspace(-5, 5, 100)
yy = torch.sigmoid(xx*a + c)

plt.plot(xx.detach().numpy(), yy.detach().numpy())
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')

plt.scatter(X_train, Y_train)

### Použitie vstavaného optimalizátora

Našťastie platí, že keď používame PyTorch, nemusíme si ručne písať vlastné optimalizačné procedúry. PyTorch má vstavaných viacero z tých najznámejších optimalizátorov. Keby sme napríklad chceli použiť optimalizátor `Adam`, jednoducho by sme ho inštancializovali pomocou tenzorov, ktoré má aktualizovať a v každej epoche by sme zavolali jeho metódu `step()`.

Prirodzene, že bude stále potrebné v každej epoche nulovať gradienty, čo sa bude teraz robiť pomocou metódy optimalizátora `zero_grad()`. Ručne nie je nutné definovať ani stredná kvadratickú chybu: PyTorch má vstavanú aj väčšinu najznámejších chybových funkcií.



In [ ]:
a = Variable(torch.as_tensor(np.random.uniform(0, 1)), requires_grad=True)
c = Variable(torch.as_tensor(np.random.uniform(0, 1)), requires_grad=True)
optimizer = torch.optim.Adam([a, c], lr=0.1)

In [ ]:
for epoch in range(500):
    optimizer.zero_grad()
    
    y = sigmoid_model(X_train_t, a, c)
    loss = torch.nn.functional.mse_loss(Y_train_t, y)
    loss.backward()
    
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch {}; loss {}.".format(epoch, loss.item()))

In [ ]:
print("a = {}\nc = {}\nloss = {}".format(
    a.item(), c.item(), loss.item()))

A znovu si môžeme overiť, ako vyzerá naša regresná krivka.



In [ ]:
xx = torch.linspace(-5, 5, 100)
yy = torch.sigmoid(xx*a + c)

plt.plot(xx.detach().numpy(), yy.detach().numpy())
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')

plt.scatter(X_train, Y_train)